In [ ]:
conda activate lenci_enviroment


In [ ]:
import sys
print(sys.executable)


In [ ]:
import sys
import surprise

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    rmse,
    mae,
    rsquared,
    exp_var,
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    get_top_k_items,
)
from recommenders.models.surprise.surprise_utils import (
    predict,
    compute_ranking_predictions,
)
from recommenders.utils.notebook_utils import store_metadata


print(f"System version: {sys.version}")
print(f"Surprise version: {surprise.__version__}")

In [ ]:
import pandas as pd
from surprise import Dataset, Reader

# Caminho do arquivo local do MovieLens 100k
file_path = "C:\\Users\\vlenc\\OneDrive\\Documentos\\ml-32m\\ratings.csv"  # ajuste para onde você extraiu o dataset

# Ler o arquivo com pandas
df = pd.read_csv(file_path, sep=",")

# Selecionar apenas as colunas necessárias
df = df[["userId", "movieId", "rating"]]

# Visualizar as primeiras linhas
print(df.head())

# Criar Dataset do Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df, reader)


   userId  movieId  rating
0       1       17     4.0
1       1       25     1.0
2       1       29     2.0
3       1       30     5.0
4       1       32     5.0


In [ ]:
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

# 1️⃣ Dividir os dados em treino e teste (75% treino, 25% teste)
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# 2️⃣ Criar o modelo SVD
svd = SVD(
    n_factors=200,   # dimensão dos vetores latentes
    n_epochs=30,     # número de iterações do SGD
    random_state=42,
    verbose=True     # imprime progresso do treino
)

# 3️⃣ Treinar o modelo
svd.fit(trainset)

# 4️⃣ Fazer previsões no conjunto de teste
predictions = svd.test(testset)

# 5️⃣ Avaliar o modelo usando RMSE e MAE
rmse_val = accuracy.rmse(predictions)
mae_val = accuracy.mae(predictions)

print(f"RMSE: {rmse_val}")
print(f"MAE: {mae_val}")


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29


In [ ]:
from collections import defaultdict
from surprise.model_selection import train_test_split

# Função auxiliar para pegar os Top-K itens recomendados
def get_top_n(predictions, n=10):
    # Dicionário para armazenar as recomendações por usuário
    user_recommendations = defaultdict(list)

    # Organiza as previsões por usuário
    for user_id, movie_id, true_rating, predicted_rating, _ in predictions:
        user_recommendations[user_id].append((movie_id, predicted_rating))

    # Ordena as predições por nota estimada e pega o Top-N
    for user_id, movie_ratings in user_recommendations.items():
        movie_ratings.sort(key=lambda x: x[1], reverse=True)
        user_recommendations[user_id] = movie_ratings[:n]

    return user_recommendations


# ⚡ Fazer previsões em todos os pares usuário-filme que não estavam no treino
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)
svd.fit(trainset)

# Gera previsões para todos os pares usuário-filme não vistos no treino
all_predictions = svd.test(trainset.build_anti_testset())

# Pega as top-10 recomendações para cada usuário
top_recommendations = get_top_n(all_predictions, n=10)

# Exibir recomendações para os 5 primeiros usuários
for user_id, recommended_movies in list(top_recommendations.items())[:5]:
    print(f"\nUsuário {user_id} - Recomendações:")
    for movie_id, predicted_rating in recommended_movies:
        print(f"  Filme {movie_id} com nota estimada {predicted_rating:.2f}")
